In [13]:
import math
import pandas as pd

In [14]:
##### user entries

min_match_len = 3

min_match_rate = .5

beg_end_str_char = "|"

value_split_char = ","

# user_match_entry = "asdkjhsehtestdfhgytrehehddoggydlgiqwertyguthd, 123234345test45qwerty65675678doggy2434534, sqwertydftestsdftestsdfdoggysdfdoggysdf, asqwertydfheuyhtestsdfkjehykuydoggysdfgsjgfhjf, sdqwertysdftestddddoggyddd, sdfgteqwertyfgftestsdfefhjdoggysdfhjf"
user_match_entry = "Discectomy, Laminectomy, Foraminotomy, Corpectomy, Spinal (Lumbar) Fusion, Spinal Cord Stimulation"

user_nope_match_entry = ''


In [15]:
def common_denominator(min_match_len: int, min_match_rate: float, beg_end_str_char: str, value_split_char: str, user_match_entry: str, user_nope_match_entry: str):

    # process user_match_entry
    var_list = user_match_entry.split(value_split_char)
    var_list = list(map(lambda x: x.strip(), var_list))
    for ind, val in enumerate(var_list):
        var_list[ind] = beg_end_str_char + var_list[ind] + beg_end_str_char
    var_list = list(map(lambda x: x.lower().strip(), var_list))
    # var_list

    # process user_no_match_entry
    var_nope_list = user_nope_match_entry.split(value_split_char)
    var_nope_list = list(map(lambda x: x.strip(), var_nope_list))
    for ind, val in enumerate(var_nope_list):
        var_nope_list[ind] = beg_end_str_char + var_nope_list[ind] + beg_end_str_char
    var_nope_list = list(map(lambda x: x.lower().strip(), var_nope_list))
    # var_nope_list

    # create the dict of all word segments
    # and a master list of all segments from all words
    var_dict = {}
    master_ref_list = set()

    for word in var_list:
        test_set = set()
        for start in range(len(word)):
            for end in range(1, len(word)+1):
                if start > end:
                    pass
                if len(word[start:end]) < min_match_len:
                    pass
                else:
                    test_set.add((word[start:end].strip()))
                    master_ref_list.add((word[start:end].strip()))
        var_dict[word] = test_set
    # master_ref_list

    # create the dict of all word segments - for nope list
    # and a master list of all segments from all words - for nope list
    var_nope_dict = {}
    master_nope_ref_list = set()

    for word in var_nope_list:
        test_set = set()
        for start in range(len(word)):
            for end in range(1, len(word)+1):
                if start > end:
                    pass
                if len(word[start:end]) < min_match_len:
                    pass
                else:
                    test_set.add((word[start:end].strip()))
                    master_nope_ref_list.add((word[start:end].strip()))
        var_nope_dict[word] = test_set
    # master_nope_ref_list

    triangle_dict = {}
    for ind, val in enumerate(var_list):
        triangle_dict[ind+1] = (ind)*(ind+1)//2
    # triangle_dict

    triangle_dict_reverse = {}
    for ind, val in enumerate(var_list):
        triangle_dict_reverse[(ind)*(ind+1)//2] = ind+1
    # triangle_dict_reverse

    min_match_number = triangle_dict[math.ceil(len(var_list)*min_match_rate)]
    # min_match_number


    # # pass 3: everything compared against the previous
    # # this should make the final full dict ranking better

    final_match_dict = {}
    for i in master_ref_list:
        final_match_dict[i] = 0
    x = 0

    for first_ind, first_val in enumerate(var_list):
        for second_ind, second_val in enumerate(var_list):
            if first_ind <= second_ind:
                pass
            else:
                for first in var_dict[var_list[first_ind]]:
                    for second in var_dict[var_list[second_ind]]:
                        x+=1
                        if first == second:
                            try:
                                final_match_dict[first] += 1
                            except:
                                pass


    final_match_list = []
    for key, value in final_match_dict.items():
        # if value == (len(var_list)-1)*len((var_list))//2:
        if value >= min_match_number:
            final_match_list.append(key)

    # print(x)

    final_match_list = sorted(final_match_list, key=len, reverse=True)

    # remove values found in the nope list
    final_match_list_temp = list(final_match_list)
    for val in final_match_list_temp:
        for nope_val in master_nope_ref_list:
            if val == nope_val:
                try:
                    final_match_list.remove(val)
                except:
                    pass
    # remove values found in the nope list
    final_match_dict_temp = final_match_dict.copy()
    for val in final_match_dict_temp:
        for nope_val in master_nope_ref_list:
            if val == nope_val:
                try:
                    del final_match_dict[val]
                except:
                    pass

    # final_match_list

    # cut out the smaller findings when there is a bigger chunk of text found
    final_match_list_temp = final_match_list.copy()
    x = 0
    for pri in final_match_list_temp:
        for sec in final_match_list_temp:
            if pri.find(sec) > -1 and pri != sec:
                try:
                    final_match_list.remove(sec)
                except:
                    x += 1

    # final_match_list

    # first main return
    # list of string parts that
        # have a match rate greater than the set minimum
        # are not part of a larger, also included string (e.g. if 'dog' is already included, 'og' will be excluded)
    # a bigger word part will however knock off a smaller word part with a higher match rate


    filtered_dict = dict()
    for (key, value) in final_match_dict.items():
        if value >= min_match_number:
            filtered_dict[key] = round(triangle_dict_reverse[value]/len(var_list),4)

    filtered_dict # maybe this one will be better to return in app?

    final_df = pd.DataFrame.from_dict(filtered_dict, orient='index')
    final_df = final_df.rename(columns={0:'Match Rate'})
    final_df = final_df.sort_values(by=['Match Rate'], ascending=False)

    # final_df

    # second main return
    # dictionary/df with all valid string parts and how often they match

    return final_match_list, final_df



In [17]:
common_denominator(3, .5, "|", ",", "Discectomy, Laminectomy, Foraminotomy, Corpectomy, Spinal (Lumbar) Fusion, Spinal Cord Stimulation", "")


(['ectomy|'],
          Match Rate
 my|          0.6667
 tomy         0.6667
 omy|         0.6667
 tomy|        0.6667
 omy          0.6667
 tom          0.6667
 ctomy        0.5000
 ecto         0.5000
 ctom         0.5000
 ectomy       0.5000
 ectom        0.5000
 cto          0.5000
 ctomy|       0.5000
 ectomy|      0.5000
 ect          0.5000)